In [6]:
import torch
import pandas as pd
from pathlib import Path
import sys
import json
import ast
import numpy as np
import matplotlib.pyplot as plt
# notebook location
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))
print("Added to path:", project_root)
from utils.general_utils import open_file, load_all_h5files_from_folder
from Model_Master import Model_Master
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from utils.I_data_preparation.experimental_config import label_to_word_map, ORIGINAL_LABELS

Added to path: /home/gspacone/Desktop/SilentWear


In [9]:
MAT_LABLES = {
        10 : "rest", 
        50: "left",
        60: "right",
        70: "up",
        80: "down",
        90: "forward",
        100: "backward",
        110: "stop",
        120: "go"
    }
original_map = MAT_LABLES
filtered_items = [(k, v) for k, v in original_map.items() if k != 10]
filtered_items

[(50, 'left'),
 (60, 'right'),
 (70, 'up'),
 (80, 'down'),
 (90, 'forward'),
 (100, 'backward'),
 (110, 'stop'),
 (120, 'go')]

In [ ]:
orig_to_train = {k: k for k in original_map.keys()}

In [ ]:
original_map = MAT_LABLES
filtered_items = [(k, v) for k, v in original_map.items() if k != 0]
filtered_items

In [15]:
original_map = ORIGINAL_LABELS


# CASE WITH REST
train_label_map = original_map.copy()
train_to_orig = {k: k for k in original_map.keys()}

orig_to_train = {k: k for k in original_map.keys()}
print("Train to origin")
print(train_to_orig)
print("Origin to train")
print(orig_to_train)
print("=============\n")
# CASE WITHOUT REST
            
filtered_items = [(k, v) for k, v in original_map.items() if k != 0]

# train labels become 0..7
train_label_map = {new_k: word for new_k, (_, word) in enumerate(filtered_items)}


train_to_orig = {
    new_k: orig_k for new_k, (orig_k, _) in enumerate(filtered_items)
}
orig_to_train = {
    orig_k: new_k for new_k, (orig_k, _) in enumerate(filtered_items)
}
print("Train to origin")
print(train_to_orig)
print("Origin to train")
print(orig_to_train)

Train to origin
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8}
Origin to train
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8}

Train to origin
{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8}
Origin to train
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7}


In [ ]:
    # remove rest (assumes rest is orig label 0)

    # # train labels become 0..7
    # self.train_label_map = {new_k: word for new_k, (_, word) in enumerate(filtered_items)}
    # print(self.train_label_map)
    
    # self.train_to_orig = {
    #     new_k: orig_k for new_k, (orig_k, _) in enumerate(filtered_items)
    # }
    # self.orig_to_train = {
    #     orig_k: new_k for new_k, (orig_k, _) in enumerate(filtered_items)
    # }


[(10, 'rest'),
 (50, 'left'),
 (60, 'right'),
 (70, 'up'),
 (80, 'down'),
 (90, 'forward'),
 (100, 'backward'),
 (110, 'stop'),
 (120, 'go')]

In [6]:

np.mean(np.array([62.25, 71.25, 68.63, 69.68, 73.80]))


np.std(np.array([62.25, 71.25, 68.63, 69.68, 73.80]))

np.float64(3.852009345783055)

In [4]:
path_model_1 = "/scratch2/gspacone/sensors_2026_speech_models/models/inter_session_ft/S01/vocalized/speechnet_base/model_1/ft_config_0/fold_1_ft_1.pt"
cpt1 = torch.load(path_model_1, weights_only=False)
path_model_2 = "/scratch2/gspacone/sensors_2026_speech_models/models/inter_session_ft/S01/vocalized/speechnet_base/model_1/ft_config_1/fold_1_ft_1.pt"
cpt2 = torch.load(path_model_2, weights_only=False)

In [34]:
from sklearn.model_selection import train_test_split

In [27]:
df_all = load_all_h5files_from_folder(Path("/baltic/users/ml_datasets/iis_bio_internal_datasets/2026_spacone_speech_classification_hmi/wins_and_feats/S01/silent"), key='wins_feats')

Found 15 .h5 files in: /baltic/users/ml_datasets/iis_bio_internal_datasets/2026_spacone_speech_classification_hmi/wins_and_feats/S01/silent


In [ ]:
df_trainval = df_all[df_all['session_id'] == 1]
# Splitting Data for Train and Validation Sets. 
# Splitting Data for Train and Validation Sets. 
train_data, val_data = train_test_split(
    df_trainval,
    test_size=0.3,
    shuffle=True,
    random_state=42,
    stratify=df_trainval["Label_int"]                  # was not here before, ensure balance of labels
)
min_samples = df_trainval['Label_int'].value_counts().min()
# downsample majority class
idx_rest = df_trainval[df_trainval['Label_str']=='rest'].index.values
index_rest_ds = df_trainval[df_trainval['Label_str']=='rest'].sample(n=min_samples, random_state=42).index.values
idx_to_drop = np.setdiff1d(idx_rest, index_rest_ds)
df_trainval = df_trainval.drop(index=idx_to_drop)

In [35]:
from models.models_factory import build_model_from_spec, ModelSpec
from models.TorchTrainer import TorchTrainer

In [27]:
model_path = r"/scratch2/gspacone/sensors_2026_speechnet_seed1/models/inter_session/S01/silent/speechnet_base/model_1/leave_one_session_out_fold_1.pt"

In [18]:
run_cfg_path = Path("/scratch2/gspacone/sensors_2026_speechnet_seed1/models/inter_session/S01/silent/speechnet_base/model_1/run_cfg.json")
run_cfg = open_file(run_cfg_path)
base_cfg = run_cfg["base_cfg"]
model_cfg = run_cfg["model_cfg"]

FS=500

In [47]:
def load_pretrained_model(base_cfg, model_cfg, pretrained_model_path):
    """Function to create a Model Master from pre-trained model"""

    num_classes = 9 if base_cfg["experiment"]["include_rest"] else 8
    spec = ModelSpec(
        kind=model_cfg["model"]["kind"],
        name=model_cfg["model"]["name"],
        kwargs=model_cfg["model"]["kwargs"], 
    )

    ctx = {
        "num_channels" : 14,                 # FIXED
        "num_samples" : int(base_cfg["window"]["window_size_s"] * FS), 
        "num_classes": num_classes,
    }

    
    model = build_model_from_spec(spec, ctx)
    cpt = torch.load(pretrained_model_path, weights_only=False)
    model.load_state_dict(cpt["model_state_dict"])

    return model


In [72]:
sum1 = pd.read_csv("/scratch2/gspacone/sensors_2026_speechnet_seed1/models/inter_session/S04/vocalized/speechnet_base/model_1/cv_summary.csv")
sum1["balanced_accuracy"].mean(), sum1["balanced_accuracy"].std()

(np.float64(0.7989814814814814), np.float64(0.05772455704114761))

In [73]:
sum2 = pd.read_csv("/scratch2/gspacone/sensors_2026_speechnet_seed1/models/inter_session_ft/S04/vocalized/speechnet_base/model_1/cv_summary.csv")
sum2["balanced_accuracy"].mean(), sum2["balanced_accuracy"].std()

(np.float64(0.8148148148148148), np.float64(0.09027834757654947))

In [76]:
ft_res = pd.read_csv("/scratch2/gspacone/sensors_2026_speechnet_seed1/models/inter_session_ft/S01/silent/speechnet_base/model_1/ft_config_0/ft_summary.csv")

In [77]:
ft_res

,subject,condition,base_model,test_session,model_to_fine_tune_name,zero_shot_test_batch,num_prev_ft_rounds,zero_shot_balanced_acc,zero_shot_cm,balanced_acc_no_ft,cm_no_ft,new_model_name
0,S01,silent,leave_one_session_out_fold_1.pt,1,leave_one_session_out_fold_1.pt,1,0,0.633333,"[[0.8, 0.15, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0....",0.633333,"[[0.8, 0.15, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0....",fold_1_ft_1.pt
1,S01,silent,leave_one_session_out_fold_1.pt,1,fold_1_ft_1.pt,2,1,0.644444,"[[0.95, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",0.522222,"[[0.85, 0.0, 0.0, 0.05, 0.1, 0.0, 0.0, 0.0, 0....",fold_1_ft_2.pt
2,S01,silent,leave_one_session_out_fold_1.pt,1,fold_1_ft_2.pt,3,2,0.711111,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...",0.688889,"[[0.95, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0....",fold_1_ft_3.pt
3,S01,silent,leave_one_session_out_fold_1.pt,1,fold_1_ft_3.pt,4,3,0.800000,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...",0.727778,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...",fold_1_ft_4.pt
4,S01,silent,leave_one_session_out_fold_1.pt,1,fold_1_ft_4.pt,5,4,0.816667,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...",0.694444,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...",fold_1_ft_5.pt
5,S01,silent,leave_one_session_out_fold_2.pt,2,leave_one_session_out_fold_2.pt,1,0,0.488889,"[[0.95, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",0.488889,"[[0.95, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",fold_2_ft_1.pt
6,S01,silent,leave_one_session_out_fold_2.pt,2,fold_2_ft_1.pt,2,1,0.638889,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...",0.444444,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...",fold_2_ft_2.pt
7,S01,silent,leave_one_session_out_fold_2.pt,2,fold_2_ft_2.pt,3,2,0.783333,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...",0.544444,"[[0.95, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0....",fold_2_ft_3.pt
8,S01,silent,leave_one_session_out_fold_2.pt,2,fold_2_ft_3.pt,4,3,0.811111,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...",0.555556,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...",fold_2_ft_4.pt
9,S01,silent,leave_one_session_out_fold_2.pt,2,fold_2_ft_4.pt,5,4,0.727778,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...",0.577778,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...",fold_2_ft_5.pt
